In [ ]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [ ]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/TGMT/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['image_40.jpg',
 'image_20.jpg',
 'image_400.jpg',
 'Colab Notebooks',
 'models',
 'FaceImage',
 'data_face_features.pickle']

In [ ]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/TGMT/data_face_features.pickle", mode="rb"))

In [ ]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.03563213  0.14579633 -0.07474371 ...  0.10712294  0.05235191
    0.01346222]]

 [[ 0.01891751  0.07340851 -0.06299383 ...  0.0674775  -0.07994623
   -0.04636956]]

 [[ 0.03296068  0.12409167  0.0214788  ...  0.13577995  0.0308777
   -0.02947379]]

 ...

 [[ 0.10115854  0.26688126 -0.05408768 ... -0.08945148  0.16420038
    0.0283456 ]]

 [[ 0.1297804   0.09853813 -0.07152011 ... -0.02761066  0.08419115
   -0.00754967]]

 [[ 0.06435601  0.00615203 -0.04063413 ...  0.11231432  0.06116884
    0.05675695]]]
['1721031660_LeQuangQuocDat' '1721031660_LeQuangQuocDat'
 '1721031660_LeQuangQuocDat' ... '162000186_NguyenVanHuong'
 '162000186_NguyenVanHuong' '162000186_NguyenVanHuong']


In [ ]:
x.shape

(4440, 1, 128)

In [ ]:
y.shape

(4440,)

In [ ]:
x = x.reshape(-1, 128)

In [ ]:
x.shape

(4440, 128)

In [ ]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3552, 128), (888, 128), (3552,), (888,))

In [ ]:
# 2. Train machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [ ]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [ ]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [ ]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.71
Accurency Test = 0.71
F1 Score Train = 0.49
F1 Score Test = 0.48


In [ ]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [ ]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.83
F1 Score Train = 0.80
F1 Score Test = 0.70


In [ ]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [ ]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.77
F1 Score Train = 1.00
F1 Score Test = 0.63


In [ ]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [ ]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [ ]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.83
F1 Score Train = 0.97
F1 Score Test = 0.70


In [ ]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [ ]:
model_best_estimator = model_grid.best_estimator_

In [ ]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   5.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   8.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   5.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   5.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   5.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   8.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gam

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [ ]:
model_grid.best_score_

0.8412162162162162

In [ ]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/TGMT/ml_face_person_identity.pkl", mode='wb'))